## Environment Setup

Before we begin, we need to configure the environment for ADK to work with Vertex AI and Google Cloud. This ensures that our agents can access the right cloud services.

These settings:
- Tell ADK to use Vertex AI for LLM requests
- Set the active GCP project and region
- No output is expected here—this just establishes the foundation

# Getting Started with Google ADK (Agent Development Kit)

Welcome to Google ADK! This notebook will introduce you to the fundamentals of building AI agents that can reason, use tools, and work together to solve complex tasks.

## What is Google ADK?

Google ADK (Agent Development Kit) is a framework for building production-ready AI agents powered by Google's Gemini models. It provides:

- **Agents**: AI-powered entities that can reason, plan, and take actions
- **Tools**: Functions that agents can call to interact with the external world
- **Orchestration**: Ways to combine multiple agents for complex workflows
- **Sessions**: Conversation management and state persistence
- **Runtime**: Infrastructure to execute agents and handle their interactions

By the end of this notebook, you'll understand:
1. How to create basic conversational agents
2. The different types of agents available in ADK
3. How to equip agents with tools
4. How to manage agent execution and conversations

Let's start by setting up our environment!

In [ ]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"
os.environ["GOOGLE_CLOUD_PROJECT"] = "traversaal-research"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

## Creating Your First Agent (No Tools!)

Let's start simple. In ADK, an **Agent** (also called **LlmAgent**) is an AI entity powered by a language model. Even without any tools, agents can:
- Have conversations
- Answer questions using their training knowledge
- Follow specific instructions and personas

Here's your first agent - a helpful assistant with a specific personality:

In [ ]:
from google.adk.agents import Agent

# Create a simple conversational agent - no tools needed!
helpful_agent = Agent(
    name="helpful_assistant",
    model="gemini-2.0-flash",
    description="A friendly and knowledgeable assistant",
    instruction="""You are a helpful assistant who loves to share interesting facts.
    When answering questions, you:
    - Are friendly and encouraging
    - Include an interesting related fact when relevant
    - Keep responses concise but informative
    """
)

print("✅ Created your first agent! No tools required.")

### Quick Test: Agent Without Tools

Before we dive deeper, let's see how this basic agent behaves. We'll set up a minimal runtime to interact with it:

> Note: We'll explain Sessions and Runners in detail later. For now, just observe how the agent responds using only its LLM capabilities.

In [ ]:
# Quick setup to test our agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

# Minimal setup (we'll explain these later)
session_service = InMemorySessionService()
await session_service.create_session(
    app_name="getting_started",
    user_id="user_1", 
    session_id="test_session"
)

runner = Runner(
    agent=helpful_agent,
    app_name="getting_started",
    session_service=session_service
)

# Ask a question - no tools involved!
content = types.Content(
    role='user', 
    parts=[types.Part(text="What's the capital of France?")]
)

print("User: What's the capital of France?\n")
async for event in runner.run_async(
    user_id="user_1", 
    session_id="test_session", 
    new_message=content
):
    if event.is_final_response() and event.content:
        print(f"Agent: {event.content.parts[0].text}")

## Understanding Different Types of Agents

ADK provides several types of agents for different purposes. According to the [official ADK documentation](https://google.github.io/adk-docs/agents/), here are the main types:

### 1. **LLM Agents (Agent/LlmAgent)**
- Powered by language models (like Gemini)
- Can reason, make decisions, and use tools
- The most common type for interactive tasks

### 2. **Workflow Agents**
These orchestrate multiple agents without doing LLM reasoning themselves:

- **Sequential Agent**: Runs sub-agents one after another
- **Parallel Agent**: Runs sub-agents simultaneously  
- **Loop Agent**: Iterates over data or repeats tasks

Let's see examples of each type!

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent

# Example 1: Sequential Agent - runs agents one after another
greeter_agent = Agent(
    name="greeter",
    model="gemini-2.0-flash",
    description="Greets the user",
    instruction="Greet the user warmly and ask how their day is going."
)

fact_agent = Agent(
    name="fact_teller", 
    model="gemini-2.0-flash",
    description="Shares an interesting fact",
    instruction="Share one interesting fact about technology or science."
)

# Sequential: First greet, then share a fact
sequential_demo = SequentialAgent(
    name="greeting_sequence",
    sub_agents=[greeter_agent, fact_agent],
    description="Greets then shares a fact"
)

print("✅ Created a Sequential Agent that will run agents in order")

In [ ]:
# Example 2: Parallel Agent - runs multiple agents at the same time
weather_reporter = Agent(
    name="weather_reporter",
    model="gemini-2.0-flash", 
    description="Reports on weather",
    instruction="Describe today's weather as sunny and pleasant."
)

news_reporter = Agent(
    name="news_reporter",
    model="gemini-2.0-flash",
    description="Reports on news",
    instruction="Share a positive headline about renewable energy."
)

# Parallel: Get weather and news simultaneously
parallel_demo = ParallelAgent(
    name="morning_briefing",
    sub_agents=[weather_reporter, news_reporter],
    description="Provides weather and news in parallel"
)

print("✅ Created a Parallel Agent that will run agents simultaneously")

### Why Different Agent Types?

Each agent type serves a specific purpose:

- **LLM Agents**: For tasks requiring reasoning, decision-making, or tool use
- **Sequential Agents**: When tasks must happen in a specific order (e.g., authenticate → fetch data → process)
- **Parallel Agents**: When tasks are independent and can run simultaneously (e.g., fetching from multiple APIs)
- **Loop Agents**: When you need to process lists or repeat operations

Now that we understand agent types, let's learn about **tools** - how agents interact with the external world!

## Introduction to Tools

While agents can have conversations using their LLM knowledge, they need **tools** to interact with the external world. According to the [ADK tools documentation](https://google.github.io/adk-docs/tools/), tools enable agents to:

- Fetch real-time data (weather, stock prices, etc.)
- Interact with APIs and databases
- Perform calculations or transformations
- Access information beyond their training data

### Types of Tools in ADK

1. **Function Tools**: Python functions you write
2. **Built-in Tools**: Pre-made tools for common tasks (BigQuery, Search, etc.)
3. **Remote Tools**: Tools hosted elsewhere (via MCP or other protocols)

Let's start with Function Tools - the simplest and most flexible type!

In [24]:
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

# A tool to get weather and time information for a city.
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (77 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


# A tool to get the current time in a specified city.
def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}




## Combining Agents with Tools

Now let's give our agent some tools! When an agent has tools:
1. It reads the tool's docstring to understand what it does
2. It decides when to use the tool based on the user's request
3. It calls the tool and processes the results
4. It formulates a response using the tool's output

Let's create an agent with weather and time tools:

In [26]:
# Define the root agent with the tools for weather and time.
weather_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "You are a helpful agent who can answer user questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)

## Understanding Sessions and Runners

Before we run our tool-equipped agent, let's understand two key ADK concepts:

### Sessions
According to the [ADK sessions documentation](https://google.github.io/adk-docs/sessions/), a **session** represents:
- A single conversation between a user and agents
- Message history and context
- Temporary state that agents can access

### Runners
From the [ADK runtime documentation](https://google.github.io/adk-docs/runtime/), a **runner**:
- Executes agents and manages their lifecycle
- Handles tool calls and event streaming
- Maintains the conversation flow

Let's set these up for our weather agent:

In [ ]:
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

### Creating a Session

Agents need context to hold a meaningful conversation—just like humans, they should remember what’s already been said or done.

In ADK, a **Session** represents a single, ongoing conversation. It tracks:

- The **message history** (`events`)
- Any **temporary state** (e.g. user inputs or results)
- Session **metadata** like user ID and app name

We use `InMemorySessionService` to create the session. This is suitable for local development and fast iteration. For production use, ADK also supports persistent session backends.

In [ ]:
from google.adk.sessions import InMemorySessionService

session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

### Creating a Runner

The `Runner` is responsible for managing the **agent execution loop**—it handles the interaction between the user, the agent, and the tools.

In this step, we initialise a `Runner` with:

- The `agent` we defined earlier  
- The `app_name` for tracking and logging  
- The `session_service` to maintain conversation context

This setup allows the runner to route inputs through the agent, trigger tool calls, and maintain state across turns—all within the defined session.

In [ ]:
from google.adk.runners import Runner

runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

## Running Your Tool-Equipped Agent

Now let's see our agent in action! We'll create a helper function to interact with the agent and observe:
- How it decides when to use tools
- The event stream showing tool calls
- How it handles successful and failed tool calls

In [1]:
# @title Define Agent Interaction Function

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

### Observing the Agent's Decision Making

In the examples below, notice how:
1. The agent analyzes the user's request
2. It decides which tool(s) to use (or none at all)
3. It handles both successful responses and errors gracefully
4. The conversation context is maintained across interactions

In [33]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("How about Paris?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # Expecting the tool's error message

    await call_agent_async("Tell me the weather in New York",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()

# --- OR ---

# Uncomment the following lines if running as a standard Python script (.py file):
# import asyncio
# if __name__ == "__main__":
#     try:
#         asyncio.run(run_conversation())
#     except Exception as e:
#         print(f"An error occurred: {e}")


>>> User Query: What is the weather like in London?


  [Event] Author: weather_time_agent, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'city': 'London'
    },
    id='adk-f92f26c8-8a65-44a3-973c-e0186aed30b3',
    name='get_weather'
  )
)] role='model'
  [Event] Author: weather_time_agent, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-f92f26c8-8a65-44a3-973c-e0186aed30b3',
    name='get_weather',
    response={
      'error_message': "Weather information for 'London' is not available.",
      'status': 'error'
    }
  )
)] role='user'
  [Event] Author: weather_time_agent, Type: Event, Final: True, Content: parts=[Part(
  text='I am sorry, I cannot get the weather in London. There was an error.'
)] role='model'
<<< Agent Response: I am sorry, I cannot get the weather in London. There was an error.

>>> User Query: How about Paris?


  [Event] Author: weather_time_agent, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'city': 'Paris'
    },
    id='adk-b08437a9-ee12-4110-906f-0d804abdc863',
    name='get_weather'
  )
)] role='model'
  [Event] Author: weather_time_agent, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-b08437a9-ee12-4110-906f-0d804abdc863',
    name='get_weather',
    response={
      'error_message': "Weather information for 'Paris' is not available.",
      'status': 'error'
    }
  )
)] role='user'
  [Event] Author: weather_time_agent, Type: Event, Final: True, Content: parts=[Part(
  text='I am sorry, I cannot get the weather in Paris. There was an error.'
)] role='model'
<<< Agent Response: I am sorry, I cannot get the weather in Paris. There was an error.

>>> User Query: Tell me the weather in New York


  [Event] Author: weather_time_agent, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'city': 'New York'
    },
    id='adk-ca9f62c5-4925-4c3a-a498-3f64b62fe819',
    name='get_weather'
  )
)] role='model'
  [Event] Author: weather_time_agent, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-ca9f62c5-4925-4c3a-a498-3f64b62fe819',
    name='get_weather',
    response={
      'report': 'The weather in New York is sunny with a temperature of 25 degrees Celsius (77 degrees Fahrenheit).',
      'status': 'success'
    }
  )
)] role='user'
  [Event] Author: weather_time_agent, Type: Event, Final: True, Content: parts=[Part(
  text='OK. The weather in New York is sunny with a temperature of 25 degrees Celsius (77 degrees Fahrenheit).'
)] role='model'
<<< Agent Response: OK. The weather in New York is sunny with a temperature of 25 degrees Celsius (77 degrees Fahrenheit).


## Key Takeaways

Congratulations! You've learned the fundamentals of Google ADK:

### 1. **Agents Come in Different Types**
- **LLM Agents**: For reasoning and tool use
- **Workflow Agents**: For orchestrating multiple agents (Sequential, Parallel, Loop)

### 2. **Tools Extend Agent Capabilities**
- Function tools are Python functions with descriptive docstrings
- Agents decide when and how to use tools based on context
- Tools can return success or error states

### 3. **Sessions and Runners Manage Execution**
- Sessions maintain conversation context
- Runners execute agents and handle the event stream
- Events show the agent's reasoning process

### 4. **Progressive Complexity**
- Start with simple conversational agents
- Add tools for external interactions
- Combine agents for complex workflows

## Next Steps

Now that you understand the basics, you can:
1. Create custom tools for your specific needs
2. Build multi-agent workflows with Sequential and Parallel agents
3. Explore built-in tools for BigQuery, Search, and more
4. Check out the [official ADK documentation](https://google.github.io/adk-docs) for advanced features

In the next notebook, we'll build a real-world data orchestration pipeline using these concepts!